In [67]:
import pandas as pd 
import geopandas as gpd
from tqdm.notebook import tqdm

In [95]:
# import datasets
gridData = gpd.read_file(r"C:\Users\nelek\OneDrive\Dokumente\Uni\Master\3. Semester\GIS\Project\Datasets\Grid_data_Wiebke\childcareBW-main\bev1kmF.geojson")

kindergartens = gpd.read_file(r"C:\Users\nelek\OneDrive\Dokumente\Uni\Master\3. Semester\GIS\Project\Datasets\combined_dataset_kindergartens.geojson")


In [96]:
gridData.head()

,ID_1km,ALTER_10JG,INSGESAMT,geometry
0,1kmN2716E4145,3.0,14.0,"POLYGON ((7.66372 47.53262, 7.66331 47.54161, ..."
1,1kmN2716E4146,52.0,351.0,"POLYGON ((7.67699 47.53290, 7.67658 47.54190, ..."
2,1kmN2716E4147,31.0,161.0,"POLYGON ((7.69025 47.53318, 7.68985 47.54218, ..."
3,1kmN2716E4148,21.0,109.0,"POLYGON ((7.70352 47.53346, 7.70312 47.54246, ..."
4,1kmN2717E4143,0.0,0.0,"POLYGON ((7.63678 47.54104, 7.63636 47.55004, ..."


In [97]:
kindergartens.head()

,id,amenity,name,geometry
0,node/25236153,kindergarten,Kindertagesstätte Pittiplatsch,POINT (8.40817 48.99901)
1,node/25243038,kindergarten,Kindergarten,POINT (8.37754 48.98666)
2,node/29793990,kindergarten,AWO-Kindertagesstätte Wiesenzwerg,POINT (8.40212 48.95010)
3,node/98071320,kindergarten,Kindertagesstätte Bernstein,POINT (8.35345 48.99722)
4,node/107597357,kindergarten,Birkenweg,POINT (9.10157 49.00583)


In [98]:
# change the projection of the data so the buffer can be inputted as meters/km
# buffer of 5000m (5km)
kindergartens['buffer'] = kindergartens['geometry'].to_crs(epsg=3035).buffer(5000).to_crs(epsg=4171)
kindergartens.head()


,id,amenity,name,geometry,buffer
0,node/25236153,kindergarten,Kindertagesstätte Pittiplatsch,POINT (8.40817 48.99901),"POLYGON ((8.47647 48.99996, 8.47627 48.99555, ..."
1,node/25243038,kindergarten,Kindergarten,POINT (8.37754 48.98666),"POLYGON ((8.44582 48.98763, 8.44563 48.98322, ..."
2,node/29793990,kindergarten,AWO-Kindertagesstätte Wiesenzwerg,POINT (8.40212 48.95010),"POLYGON ((8.47035 48.95106, 8.47016 48.94665, ..."
3,node/98071320,kindergarten,Kindertagesstätte Bernstein,POINT (8.35345 48.99722),"POLYGON ((8.42174 48.99820, 8.42156 48.99379, ..."
4,node/107597357,kindergarten,Birkenweg,POINT (9.10157 49.00583),"POLYGON ((9.16989 49.00636, 9.16964 49.00195, ..."


In [99]:
# build subsets
k = kindergartens
g = gridData[0:200]

# add new empty columns to dataframes
k["sum_children"] = 0

k["sum_total_population"] = 0

k["grid_cell_IDs_in_buffer"] = [[] for _ in range(len(k))]

g["nr_kiga_in_reach"] = 0

g["kiga_IDs_in_reach"] =  [[] for _ in range(len(g))]


c:\Users\nelek\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [103]:
k.head()

,id,amenity,name,geometry,buffer,sum_children,sum_total_population,grid_cell_IDs_in_buffer
0,node/25236153,kindergarten,Kindertagesstätte Pittiplatsch,POINT (8.40817 48.99901),"POLYGON ((8.47647 48.99996, 8.47627 48.99555, ...",0,0,[]
1,node/25243038,kindergarten,Kindergarten,POINT (8.37754 48.98666),"POLYGON ((8.44582 48.98763, 8.44563 48.98322, ...",0,0,[]
2,node/29793990,kindergarten,AWO-Kindertagesstätte Wiesenzwerg,POINT (8.40212 48.95010),"POLYGON ((8.47035 48.95106, 8.47016 48.94665, ...",0,0,[]
3,node/98071320,kindergarten,Kindertagesstätte Bernstein,POINT (8.35345 48.99722),"POLYGON ((8.42174 48.99820, 8.42156 48.99379, ...",0,0,[]
4,node/107597357,kindergarten,Birkenweg,POINT (9.10157 49.00583),"POLYGON ((9.16989 49.00636, 9.16964 49.00195, ...",0,0,[]


In [104]:
g.head()
#print(g.iloc[0, 3])

,ID_1km,ALTER_10JG,INSGESAMT,geometry,nr_kiga_in_reach,kiga_IDs_in_reach
0,1kmN2716E4145,3.0,14.0,"POLYGON ((7.66372 47.53262, 7.66331 47.54161, ...",1,[node/1614778817]
1,1kmN2716E4146,52.0,351.0,"POLYGON ((7.67699 47.53290, 7.67658 47.54190, ...",2,"[node/1614778817, way/148502012]"
2,1kmN2716E4147,31.0,161.0,"POLYGON ((7.69025 47.53318, 7.68985 47.54218, ...",0,[]
3,1kmN2716E4148,21.0,109.0,"POLYGON ((7.70352 47.53346, 7.70312 47.54246, ...",3,"[node/309885883, way/126171180, way/126171181]"
4,1kmN2717E4143,0.0,0.0,"POLYGON ((7.63678 47.54104, 7.63636 47.55004, ...",4,"[way/102724057, way/107104065, way/107106174, ..."


In [102]:
# look for grid-polygons that lie in the buffer zone
# iterrows is too slow?!

for index1, row1 in tqdm(k.iterrows()):
    
    for index2, row2 in g.iterrows():
        
        if(row2[3].overlaps(row1[4])):
            # sum_children for each kindergarten-buffer
            k.iloc[index1, 5] =  row1[5] + row2[1]

            # sum_total_population for each kindergarten-buffer
            k.iloc[index1, 6] = row1[6] + row2[2]

            # grid_cell_IDs_in_buffer (add all grid-cell-IDs that are in each kindergarten buffer)
            if row2[0] not in row1[7]:
                 k.iloc[index1, 7].append(row2[0])
            
            # nr_kiga_in_reach of each grid-cell
            g.iloc[index2, 4] += 1

            # kiga_IDs_in_reach (all kindergarten-IDs in reach of the grid-cell)
            if row1[0] not in row2[5]:
                g.iloc[index2, 5].append(row1[0])

0it [00:00, ?it/s]

c:\Users\nelek\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [108]:
k["sum_children"].unique()

array([  0,   3, 167,  86,  23, 120,  37, 124,  93,   6,  42,  43,  46,
       371, 284,  22,  17,  45, 272,  75, 104], dtype=int64)

In [106]:
g["nr_kiga_in_reach"].unique()

array([ 1,  2,  0,  3,  4,  5,  8,  7, 10, 12,  6,  9, 15, 11, 14, 16, 17,
       13], dtype=int64)

In [ ]:
k.to_file(r'C:\Users\nelek\OneDrive\Dokumente\Uni\Master\3. Semester\GIS\Project\kindergartens_final.geojson', driver='GeoJSON')

In [ ]:
g.to_file(r'C:\Users\nelek\OneDrive\Dokumente\Uni\Master\3. Semester\GIS\Project\gridData_final.geojson', driver='GeoJSON')